In [167]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [168]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

The number of calendar data is 20.


1. Loading the dataset and json
2. Prompt the dataset column and json question to gpt-4 to generate the python function
3. pass the generated funtion and dataframe into the test function
4. save the result into json-ansewer

In [169]:
question_index = 1 # for question test

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    print(json_data[question_index])

{'question': 'How many events do I have scheduled for today?', 'answer': ''}


In [170]:
print(f"The total number of the question is {len(json_data)}.")
print(f"Selected question is: <{json_data[question_index]['question']}>.")

The total number of the question is 8.
Selected question is: <How many events do I have scheduled for today?>.


In [171]:
text = f"""
{json_data[question_index]['question']}
"""
prompt = f"""You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python function object without any prefix instructionn and explaination.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by '\n' with 1 sentence.

Output using the folowing format:
code language: python
function name: query():
input: calendar_data
output is a python function without any prefix instructionn and explaination.

Today is 2024-03-12 13:02:30-03:20

For example, the output only have the following format:
Sample 1:
def query(calendar_data):
    return calendar_data[0]

Question to be resolved: {text} 
"""

In [172]:
print(prompt)

You are provided a DataFrame ```calendar_data(ID, status, summary, start, end , duration, attendees)```. This dataframe includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python function object without any prefix instructionn and explaination.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss-00:00, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by '
' with 1 sentence.

Output using the folowing format:
code language: python
function name: query():
input: calendar_data
output is a python function without any prefix instruc

In [173]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model=MODEL,
)

# verify the output
generated_text = chat_completion.choices[0].message.content
print(generated_text)

def query(calendar_data):
    today_date = "2024-03-12"
    today_events = calendar_data[(calendar_data['start'].str.contains(today_date)) & (calendar_data['status'] == 'confirmed')].count()
    return today_events['ID']


In [182]:
fp = open('generated_code.py', 'w')
fp.write(generated_text)
fp.close()

In [188]:
from generated_code import *
answer = query(calendar_data)
print(answer)
# try:
#     print(json_data[question_index]['question']) 
#     answer = query(calendar_data)
#     print(answer)
# except:
#     print("An exception occurred")

20


In [184]:

def query(calendar_data):
    today_date = "2024-03-12"
    today_events = calendar_data[(calendar_data['start'].str.contains(today_date)) & (calendar_data['status'] == 'confirmed')].count()
    return today_events['ID']
answer = query(calendar_data)
print(answer)

2


Save the result into the .json file

In [84]:
print(json_data)

[{'question': 'How many meetings do I have scheduled in total?', 'answer': ''}, {'question': 'How many events do I have scheduled for today?', 'answer': ''}, {'question': 'How many attendees are there for the meeting with ID masbk72a24cb0a8k9c7jo0e9s6?', 'answer': ''}, {'question': 'What is the longest meeting ID on my calendar?', 'answer': ''}, {'question': 'List events with duration longer than 2 hours?', 'answer': ''}, {'question': "Find events with the status 'cancelled'?", 'answer': ''}, {'question': 'What events are scheduled to start tomorrow?', 'answer': ''}, {'question': 'What is the status and summary of the meeting with ID maeuv72w37ab0p6m8n7io4r3x6?', 'answer': ''}]



### Error

1 "\n"
    should be "\\n"
2 Date
    unexpcted prefix
2. python package, eg 
    def query():
        import panda as pd
